## Accessing Sentinel-1 RTC data with the Planetary Computer STAC API

The [Sentinel 1 RTC](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc) product in this collection is a radiometrically terrain corrected product derived from the [Sentinel-1 Ground Range Detected (GRD)](https://planetarycomputer.microsoft.com/dataset/sentinel-1-grd) Level-1 products produced by the European Space Agency.

### Environment setup

Running this notebook requires an API key.

* The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) is pre-configured to use your API key.
* To use your API key locally, set the environment variable `PC_SDK_SUBSCRIPTION_KEY` or use `planetary_computer.settings.set_subscription_key(<YOUR API Key>)`

See [when an account is needed](https://planetarycomputer.microsoft.com/docs/concepts/sas/#when-an-account-is-needed) for more, and [request an account](http://planetarycomputer.microsoft.com/account/request) if needed.




### TODO
- [x] group by solar day
- [ ] dont load data no 90% clear - might be a problem, keep investigating
- [ ] why is distribution not bimodel - not too fussed at the moment
- [x] make sum of time series
- [x] make count of time series
- [x] make frequency of time series
- [x] export as geotiff


Notebook taken from here
https://docs.dea.ga.gov.au/notebooks/Real_world_examples/Radar_water_detection/

In [7]:
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pystac
import pystac_client
import planetary_computer
import requests
import rich.table

import odc.stac
from datacube.utils.cog import write_cog

from scipy.ndimage import uniform_filter
from scipy.ndimage import variance

from IPython.display import Image

### Data access

The datasets hosted by the Planetary Computer are available from [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more.

In [8]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

### Choose an area and time of interest

We'll search for assets acquired over Panama in the first week of May, 2022. You can use the [Planetary Computer Explorer](https://planetarycomputer.microsoft.com/explore?c=-79.6735%2C9.0461&z=9.91&ae=0&v=2&d=sentinel-1-rtc&s=false%3A%3A100%3A%3Atrue&m=Most+recent+-+VV%2C+VH&r=VV%2C+VH+False-color+composite) to find areas of interest.

In [9]:
site_name = 'Grafton'
time = "2022-03-01/2022-03-05"
bbox = [150.6839, -33.7077, 153.3037, -28.7818,] # lismore

# site_name = 'dea_example'
# time = "2017-01-01/2017-03-31"
# bbox = [130.324, -11.288, 130.453, -11.086] # dea example

starttime, endtime = time.split('/')
search = catalog.search(
    collections=["sentinel-1-rtc"], bbox=bbox, datetime=time # eden
    # collections=["sentinel-1-rtc"], bbox=bbox, datetime=time # dea example
)
items = search.item_collection()
print(f"Found {len(items)} items")
item = items[0]

Found 6 items


In [10]:
items


id: S1A_IW_GRDH_1SSH_20220305T083245_20220305T083314_042183_0506E1_rtc
"bbox: [150.06790863, -29.46306217, 153.02345511, -27.1447651]"
datetime: 2022-03-05T08:33:00.410058Z
platform: SENTINEL-1A
"s1:shape: [29806, 26062]"
"proj:bbox: [208900.0, 6737330.0, 506960.0, 6997950.0]"
proj:epsg: 32756
"proj:shape: [26062, 29806]"
end_datetime: 2022-03-05 08:33:14.921481+00:00
constellation: Sentinel-1
s1:resolution: high


In [ ]:
%%time
# load as odc stac dataset
ds_s1 = odc.stac.load(items,
                        bbox=bbox,
                        crs="EPSG:3577",
                        resolution=10,
                        groupby='solar_day')

In [ ]:
ds_s1

In [ ]:
#select_pixel = ds_s1.isel(time=0)
#select_pixel

In [ ]:
#REMOVE TIME SLICE 1
# Assuming ds is your xarray dataset

# Remove time=1 from the dataset
#ds_s1_removed_time1 = ds_s1.sel(time=~(ds_s1.time == ds_s1.time[1]))




In [ ]:
#ds_s1_removed_time1

In [ ]:
#ds_s1 = ds_s1_removed_time1

In [ ]:
# select_pixel = ds_s1.isel(x=2, y=9, time=0)
#select_pixel

In [ ]:
# Scale to plot data in decibels
ds_s1["vh_dB"] = 10 * np.log10(ds_s1.vv)

# Plot all VH observations for the year
ds_s1.vh_dB.plot(cmap="Greys_r", robust=True,  col="time", col_wrap=5)
plt.show()

In [ ]:
ds_s1["vh_dB"]

In [ ]:
# PLOTTING SINGLE TIME SLICE

# Plot the "vh_dB" variable directly
#fig = plt.figure(figsize=(6, 4))
#ds_s1.vh_dB.plot(cmap="Greys_r", robust=True)

#plt.title("VH_dB")
#plt.show()


In [ ]:
# Plot the average of all VH observations
mean_vh_dB = ds_s1.vh_dB.mean(dim="time")

fig = plt.figure(figsize=(6, 4))
mean_vh_dB.plot(cmap="Greys_r", robust=True)
plt.title("Average VV")
plt.show()

In [ ]:
mean_vh_dB.max()

In [ ]:
# Scale to plot data in decibels
ds_s1["vv_dB"] = 10 * np.log10(ds_s1.vv)

# Plot all VV observations for the year
ds_s1.vv_dB.plot(cmap="Greys_r", robust=True,  col="time", col_wrap=5)
plt.show()

In [ ]:
# Plot the average of all VV observations
mean_vv_dB = ds_s1.vv_dB.mean(dim="time")

fig = plt.figure(figsize=(6, 4))
mean_vv_dB.plot(cmap="Greys_r", robust=True)
plt.title("Average VV")
plt.show()

In [ ]:
# Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python
def lee_filter(da, size):
    img = da.values
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

In [ ]:
# Set any null values to 0 before applying the filter to prevent issues
ds_s1_filled = ds_s1.where(~ds_s1.isnull(), 0)

# Create a new entry in dataset corresponding to filtered VV and VH data
ds_s1["filtered_vv"] = ds_s1_filled.vv.groupby("time").apply(lee_filter, size=7)
ds_s1["filtered_vh"] = ds_s1_filled.vh.groupby("time").apply(lee_filter, size=7)

In [ ]:
# Scale to plot data in decibels
ds_s1["filtered_vh_dB"] = 10 * np.log10(ds_s1.filtered_vh)

# Plot all filtered VH observations for the year
ds_s1.filtered_vh_dB.plot(cmap="Greys_r", robust=True, col="time", col_wrap=5)
plt.show()

In [ ]:
# Plot the average of all filtered VH observations
mean_filtered_vh_dB = ds_s1.filtered_vh_dB.mean(dim="time")

fig = plt.figure(figsize=(6, 4))
mean_filtered_vh_dB.plot(cmap="Greys_r", robust=True)
plt.title("Average filtered VH")
plt.show()

In [ ]:
# Scale to plot data in decibels
ds_s1["filtered_vv_dB"] = 10 * np.log10(ds_s1.filtered_vv)

# Plot all filtered VV observations for the year
ds_s1.filtered_vv_dB.plot(cmap="Greys_r", robust=True, col="time", col_wrap=5)
plt.show()

In [ ]:
# Plot the average of all filtered VV observations
mean_filtered_vv_dB = ds_s1.filtered_vv_dB.mean(dim="time")

fig = plt.figure(figsize=(6, 4))
mean_filtered_vv_dB.plot(cmap="Greys_r", robust=True)
plt.title("Average filtered VV")
plt.show()

## VH

In [ ]:
fig = plt.figure(figsize=(15, 3))
ds_s1.filtered_vh_dB.plot.hist(bins=1000, label="VH filtered")
ds_s1.vh_dB.plot.hist(bins=1000, label="VH", alpha=0.5)
plt.legend()
plt.xlabel("VH (dB)")
plt.title("Comparison of filtered VH bands to original")
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 3))
ds_s1.filtered_vv_dB.plot.hist(bins=1000, label="VV filtered")
ds_s1.vv_dB.plot.hist(bins=1000, label="VV", alpha=0.5)
plt.legend()
plt.xlabel("VV (dB)")
plt.title("Comparison of filtered VV bands to original")
plt.show()

In [ ]:
threshold = -20

In [ ]:
fig = plt.figure(figsize=(15, 3))
plt.axvline(x=threshold, label=f"Threshold at {threshold}", color="red")
ds_s1.filtered_vh_dB.plot.hist(bins=1000, label="VH filtered")
ds_s1.vh_dB.plot.hist(bins=1000, label="VH", alpha=0.5)
plt.legend()
plt.xlabel("VH (dB)")
plt.title("Histogram Comparison of filtered VH bands to original")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))
ds_s1.filtered_vh_dB.plot.hist(bins=1000, label="VH filtered")
ax.axvspan(xmin=-40.0, xmax=threshold, alpha=0.25, color="green", label="Water")
ax.axvspan(xmin=threshold,
           xmax=-0.5,
           alpha=0.25,
           color="red",
           label="Not Water")
plt.legend()
plt.xlabel("VH (dB)")
plt.title("Effect of the classifier")
plt.show()

In [ ]:
def s1_water_classifier(ds, threshold=-20.0):
    assert "vh" in ds.data_vars, "This classifier is expecting a variable named `vh` expressed in DN, not DB values"
    filtered = ds.vh.groupby("time").apply(lee_filter, size=7)
    water_data_array = 10 * np.log10(filtered) < threshold
    return water_data_array.to_dataset(name="s1_water")

In [ ]:
ds_s1["water"] = s1_water_classifier(ds_s1).s1_water

In [ ]:
print(ds_s1.water)

In [ ]:
# Plot the mean of each classified pixel value
plt.figure(figsize=(15, 12))
ds_s1_mean = ds_s1.water.mean(dim="time")
ds_s1_mean.plot(cmap="RdBu")
plt.title("Average classified pixel value")
plt.show()

In [ ]:
#WRITECOG SINGLE TIME SLICE
select_pixel = ds_s1.water.isel(time=0)

select_pixel.attrs = ds_s1_mean.attrs

In [ ]:
select_pixel


In [ ]:
# Assuming 'select_pixel' is your xarray Dataset
variable_to_write = select_pixel.astype(int)

# Write GeoTIFF of frequency
write_cog(
    geo_im=variable_to_write,
    fname='/home/jovyan/code/Flooding_Sent_1/data/' + site_name + '_' + starttime + '_' + endtime + '_vh.tif',
    overwrite=True,
    nodata=0
)


In [ ]:
# Assuming select_pixel is your xarray Dataset
variable_to_write = select_pixel

# Write GeoTIFF of frequency
write_cog(
    geo_im=variable_to_write,
    fname='code/Flooding_Sent_1/data/' + site_name + '_' + starttime + '_' + endtime + '_vh.tif',
    overwrite=True,
    nodata=0.0
)


In [ ]:
# Write GeoTIFF of frequency ##### remember doesn't overwrite so need to delete if already exists #####
write_cog(geo_im=select_pixel,
          fname='./data/' + site_name + '_' + starttime + '_' + endtime + '.tif',
          overwrite=True,
           nodata=0.0
         )

### VV

In [ ]:
threshold = -12.0

In [ ]:
fig = plt.figure(figsize=(15, 3))
plt.axvline(x=threshold, label=f"Threshold at {threshold}", color="red")
ds_s1.filtered_vv_dB.plot.hist(bins=1000, label="VV filtered")
ds_s1.vv_dB.plot.hist(bins=1000, label="VV", alpha=0.5)
plt.legend()
plt.xlabel("VV (dB)")
plt.title("Histogram Comparison of filtered VH bands to original")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))
ds_s1.filtered_vv_dB.plot.hist(bins=1000, label="VV filtered")
ax.axvspan(xmin=-40.0, xmax=threshold, alpha=0.25, color="green", label="Water")
ax.axvspan(xmin=threshold,
           xmax=-0.5,
           alpha=0.25,
           color="red",
           label="Not Water")
plt.legend()
plt.xlabel("VV (dB)")
plt.title("Effect of the classifier")
plt.show()

In [ ]:
def s1_water_classifier(ds, threshold=-20.0):
    assert "vv" in ds.data_vars, "This classifier is expecting a variable named `vv` expressed in DN, not DB values"
    filtered = ds.vv.groupby("time").apply(lee_filter, size=7)
    water_data_array = 10 * np.log10(filtered) < threshold
    return water_data_array.to_dataset(name="s1_water")

In [ ]:
ds_s1["water"] = s1_water_classifier(ds_s1).s1_water

In [ ]:
print(ds_s1.water)

In [ ]:
# Plot the mean of each classified pixel value
plt.figure(figsize=(15, 12))
ds_s1_mean = ds_s1.water.mean(dim="time")
ds_s1_mean.plot(cmap="RdBu")
plt.title("Average classified pixel value")
plt.show()

In [ ]:
# Write GeoTIFF of frequency ##### remember doesn't overwrite so need to delete if already exists #####
write_cog(geo_im=ds_s1_mean,
          fname='./data/' + site_name + '_' + starttime + '_' + endtime + '.tif',
          overwrite=True,
          # nodata=0.0
         )